In [1]:
from line_solver import *
GlobalConstants.set_verbose(VerboseLevel.STD)

In [2]:
model = Network('model')

# Block 1: nodes
node1 = Delay(model, "Delay")
node2 = Queue(model, "Queue1", SchedStrategy.FCFS)
node3 = Queue(model, "Queue2", SchedStrategy.FCFS)
node2.set_number_of_servers(3)
node3.set_number_of_servers(3)

# Block 2: classes
jobclass1 = ClosedClass(model, "Class1", 2, node1, 0)
jobclass2 = ClosedClass(model, "Class2", 2, node1, 0)
jobclass3 = ClosedClass(model, "Class3", 2, node1, 0)
jobclass4 = ClosedClass(model, "Class4", 1, node1, 0)

node1.set_service(jobclass1, Exp(1)) # (Delay,Class1)
node1.set_service(jobclass2, Exp(1)) # (Delay,Class2)
node1.set_service(jobclass3, Exp(10)) # (Delay,Class3)
node1.set_service(jobclass4, Exp(1)) # (Delay,Class4)

node2.set_service(jobclass1, Exp(1)) # (Queue1,Class1)
node2.set_service(jobclass2, Erlang(1, 2)) # (Queue1,Class2)
node2.set_service(jobclass3, Exp(10)) # (Queue1,Class3)
node2.set_service(jobclass4, Exp(1)) # (Queue1,Class4)

node3.set_service(jobclass1, Disabled()) # (Queue2,Class1)
node3.set_service(jobclass2, Disabled()) # (Queue2,Class2)
node3.set_service(jobclass3, Erlang(1, 2)) # (Queue2,Class3)
node3.set_service(jobclass4, Exp(1)) # (Queue2,Class4)

# Block 3: topology
routingMatrix = model.init_routing_matrix()

routingMatrix.set(jobclass1, jobclass1, node1, node2, 0.50)
routingMatrix.set(jobclass1, jobclass1, node2, node1, 1.00)
routingMatrix.set(jobclass1, jobclass1, node3, node1, 1.00)

routingMatrix.set(jobclass1, jobclass2, node1, node2, 0.50)

routingMatrix.set(jobclass2, jobclass2, node2, node1, 1.00)
routingMatrix.set(jobclass2, jobclass2, node3, node1, 1.00)

routingMatrix.set(jobclass2, jobclass1, node1, node2, 1.00)

routingMatrix.set(jobclass3, jobclass3, node1, node2, 0.25)
routingMatrix.set(jobclass3, jobclass3, node1, node3, 0.25)
routingMatrix.set(jobclass3, jobclass3, node2, node1, 1.00)
routingMatrix.set(jobclass3, jobclass3, node3, node1, 1.00)

routingMatrix.set(jobclass3, jobclass4, node1, node2, 0.50)

routingMatrix.set(jobclass4, jobclass4, node2, node1, 1.00)
routingMatrix.set(jobclass4, jobclass4, node3, node1, 1.00)

routingMatrix.set(jobclass4, jobclass3, node1, node2, 1.00)

model.link(routingMatrix)

# Create nodes array for State operations
node = np.array([node1, node2, node3])

# State operations like MATLAB - using node index 1 (second node, node2)
spaceRunning = State.fromMarginalAndRunning(model, 1, [2,1,1,1], [2,1,0,0])
spaceStarted = State.fromMarginalAndStarted(model, 1, [2,1,1,1], [2,1,0,0])
space = State.fromMarginal(model, 1, [2,1,1,1])

In [3]:
# This example demonstrates State operations as shown in MATLAB
print("spaceRunning:", spaceRunning)
print("spaceStarted:", spaceStarted)
print("space:", space)

spaceRunning: [[4. 3. 2. 1. 0. 0. 0.]
 [4. 3. 2. 0. 1. 0. 0.]
 [3. 4. 2. 1. 0. 0. 0.]
 [3. 4. 2. 0. 1. 0. 0.]]
spaceStarted: [[4. 3. 2. 1. 0. 0. 0.]
 [3. 4. 2. 1. 0. 0. 0.]]
space: [[4. 3. 2. 1. 0. 0. 0.]
 [4. 3. 2. 0. 1. 0. 0.]
 [4. 2. 2. 0. 0. 1. 0.]
 [4. 1. 1. 1. 0. 1. 0.]
 [4. 1. 1. 0. 1. 1. 0.]
 [3. 4. 2. 1. 0. 0. 0.]
 [3. 4. 2. 0. 1. 0. 0.]
 [3. 2. 2. 0. 0. 0. 1.]
 [3. 1. 1. 1. 0. 0. 1.]
 [3. 1. 1. 0. 1. 0. 1.]
 [2. 4. 2. 0. 0. 1. 0.]
 [2. 3. 2. 0. 0. 0. 1.]
 [2. 1. 1. 0. 0. 1. 1.]
 [1. 4. 1. 1. 0. 1. 0.]
 [1. 4. 1. 0. 1. 1. 0.]
 [1. 3. 1. 1. 0. 0. 1.]
 [1. 3. 1. 0. 1. 0. 1.]
 [1. 2. 1. 0. 0. 1. 1.]
 [1. 1. 0. 1. 0. 1. 1.]
 [1. 1. 0. 0. 1. 1. 1.]]
